In [ ]:
import time

import sc3nb as scn
from sc3nb import Synth

In [ ]:
sc = scn.startup(with_blip=False)

# Nodes (Synth and Group)

One of the most important objects in SuperCollider are Nodes.

To see all Nodes on the Server use

In [ ]:
sc.server.dump_tree()

You can also get a Python object represenation of the current server state via.

In [ ]:
root_node = sc.server.query_tree()
root_node

Note that this will send a `/g_queryTree` command and parse the `/g_queryTree.reply` response of the server. The resulting Group representation will currently not be updated unless you use `query_tree` again.

In [ ]:
root_node.children[-1]

A node is either a [Synth](#Synth) or a [Group](#Group) of nodes

## Synth

### Create and control a Synth

Create a new Synth

In [ ]:
synth = Synth(name="s2", controls={"freq": 100})
synth

You can now hear the Synth playing and see it in the NodeTree

In [ ]:
sc.server.default_group

In [ ]:
sc.server.query_tree()

Free a synth

In [ ]:
synth.free()

Start the synth again

In [ ]:
synth.new()

In [ ]:
synth.nodeid

Calling `new` on an already running synth will cause a SuperCollider Server error

In [ ]:
synth.new()
time.sleep(0.5)  # wait some time to ensure the audio server has send the /fail OSC message

Pause a synth

In [ ]:
synth.run(False)

In [ ]:
synth

Run a paused synth

In [ ]:
synth.run() # default flag for run is True

You can also wait for a Synth

In [ ]:
synth_with_duration = scn.Synth("s1", dict(dur=2))
# wait for the Synth to finish playing
synth_with_duration.wait()

### Set / get Synth parameters

Set synth parameters, using any for the following `set` calls

    set(key, value, ...)
    set(list_of_keys_and_values])
    set(dict)

In [ ]:
synth.set("freq", 200)

In [ ]:
synth.set(["freq", 30, "amp", 0.3])

In [ ]:
synth.set({"freq": 130, "amp": 0.1})

The Synth does save its current control arguments in `current_controls`

In [ ]:
synth.current_controls

However these are only cached values on the python object. 
Updating them will only affect the `new` call. See below why this can be useful.

In [ ]:
synth.current_controls['freq'] = 440
synth

In [ ]:
synth.free()
synth.new() # The new Synth will be created with freq = 440

To see what arguments can be set you can look at the `synth_desc`

In [ ]:
synth.synth_desc

You can use ``get`` to see the current value. This will request the current value from the SuperCollider audio server

In [ ]:
synth.get("freq")

In [ ]:
synth.get("pan")

This will also update the cached values in `current_args`

In [ ]:
synth

This is also possible directly with

In [ ]:
synth.pan

Which can also be used for setting the argument

In [ ]:
synth.pan = -1 

You can also query information about the Synth. Look at [Group](#Group) for more information about these values

In [ ]:
synth.query()

In [ ]:
synth.free()

Keep in mind that getting a synth value is always querying the server.

This means we need to receive a message which cant be done using the Bundler.
If you want to use relative values in a Bundler you should use the cached values from `current_args`

In [ ]:
with sc.server.bundler() as bundle:
    synth.new({"freq": 600})
    for _ in range(100):
        synth.set(['freq', synth.current_controls['freq'] * 0.99])
        bundle.wait(0.05)
    synth.free()

In [ ]:
synth.wait() # wait for synth

Some methods also allow getting the OSC Message instead of sending

In [ ]:
synth.new(return_msg=True)

Refer to the [OSC communication example notebook](../osc-communication-examples.ipynb) if you want to learn more about messages and bundles.

## Group

Nodes can be grouped and controlled together. This gives you mutliple advantages like

* controlling multiple Synth together
* specifying the execution order of the nodes. For more details look at [Order of nodes](#Order-of-Nodes)

The SuperCollider audio server scsynth does have one root Group with the Node ID 0.

In this root group each user got an default group were all the Nodes of the user (Synths and Groups) should be located

In [ ]:
sc.server.query_tree()

We have the following default Group

In [ ]:
sc.server.default_group

### Creating Groups

In [ ]:
g0 = scn.Group()
g0

In [ ]:
sc.server.dump_tree()

In [ ]:
g0.free()

In [ ]:
sc.server.dump_tree()

In [ ]:
g0.new()

In [ ]:
sc.server.query_tree()

In [ ]:
sc.server.default_group

Create a Group in our new Group

In [ ]:
g1 = scn.Group(target=g0)
g1

In [ ]:
sc.server.query_tree()

You can get information about you Group via

In [ ]:
g0.query()

The query contains the same information as a Synth query and additionally the head and tail Node of this group.

Notice the special value `-1` meaning `None`.

In [ ]:
g1.query()

Free the node

In [ ]:
g0.free()

In [ ]:
sc.server.query_tree()

## Order of Nodes 



The execution of the Nodes does plays an important role. 
For more details look at the [SuperCollider Documentation](http://doc.sccode.org/Guides/Order-of-execution.html)

The Node placement of a Nodes can be controlled by the instantiation arguments

* `add_action` - An AddAction that specifies where to put Node relative to the target

In [ ]:
list(scn.AddAction)

* `target` - The target of the AddAction
* `group` - The group were the Node will be placed

### Example of Ordering Nodes

In [ ]:
g0 = scn.Group()
g0

In [ ]:
g1 = scn.Group(target=g0)
g1

In [ ]:
s0 = scn.Synth(target=g0, controls={"freq": 200})

In [ ]:
s1 = scn.Synth(add_action=scn.AddAction.BEFORE, target=s0, controls={"freq": 600})

In [ ]:
s2 = scn.Synth(add_action=scn.AddAction.TO_TAIL, target=g1, controls={"freq": 1200})

In [ ]:
sc.server.query_tree()

In [ ]:
s1.move(scn.AddAction.BEFORE, s2)

In [ ]:
sc.server.query_tree()

In [ ]:
g0

In [ ]:
g0.run(False)

In [ ]:
g0.run(True)

In [ ]:
g1.run(False)
g1.query_tree()

In [ ]:
s0.freq, s1.freq, s2.freq

In [ ]:
g0.set("freq", 100)
s0.freq, s1.freq, s2.freq

In [ ]:
sc.server.dump_tree()

In [ ]:
g1.run(True)

In [ ]:
sc.server.query_tree()

In [ ]:
g1.set("freq", 440)

s0.freq, s1.freq, s2.freq

In [ ]:
g0.query_tree()

In [ ]:
g0.move_node_to_tail(s1)

In [ ]:
g0.dump_tree()

In [ ]:
g1.set("freq", 800)

s0.freq, s1.freq, s2.freq

In [ ]:
sc.server.default_group.query_tree()

In [ ]:
sc.server.free_all()
sc.server.dump_tree()

In [ ]:
sc.exit()